## 20110342_Nguyễn Thanh Trúc

In [1]:
#cài đặt thư viện
!pip install -q lightning

In [2]:
!pip install torch

In [3]:
import torch
import lightning as L
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os

In [6]:
os.getcwd()

'/content'

In [7]:
os.chdir("/content/drive/MyDrive/Colab Notebooks")

In [8]:
data = pd.read_csv("iris.csv")
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [9]:
#encode cho label
labels = {}
for index, element in enumerate(data["variety"].unique()):
  labels[element] = index

data.loc[:, "labels"] = data["variety"].apply(lambda x: labels[x])
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety,labels
0,5.1,3.5,1.4,0.2,Setosa,0
1,4.9,3.0,1.4,0.2,Setosa,0
2,4.7,3.2,1.3,0.2,Setosa,0
3,4.6,3.1,1.5,0.2,Setosa,0
4,5.0,3.6,1.4,0.2,Setosa,0


In [10]:
#tạo class Dataset để đọc data
COLUMNS = data.drop(['variety', "labels"], axis= 1).columns.to_list()

class DataSet(torch.utils.data.Dataset):
    def __init__(self, data, normalizer, columns=COLUMNS):
        super(DataSet, self).__init__()
        self.data = data
        self.feature = normalizer.transform(self.data[columns].values)
        self.feature = torch.tensor(self.feature).float()
        self.label = torch.tensor(self.data["labels"].values)

    def __len__(self):
        return len(self.feature)

    def __getitem__(self, idx):
        return {"feature": self.feature[idx],
                "label": self.label[idx]}


In [11]:
#Chia và chuẩn hóa dataset
from sklearn.preprocessing import StandardScaler
BATCH_SIZE = 256
train_data, test_data = train_test_split(data, test_size=0.2, random_state=0)
test_data, val_data = train_test_split(train_data, test_size=0.25, random_state=0)
normalizer = StandardScaler()
normalizer.fit(train_data[COLUMNS].values)

train_loader = torch.utils.data.DataLoader(DataSet(train_data, normalizer), batch_size=BATCH_SIZE,
                                           shuffle=True, num_workers=2, pin_memory=True)
val_loader = torch.utils.data.DataLoader(DataSet(val_data, normalizer), batch_size=BATCH_SIZE, num_workers=2,pin_memory=True)
test_loader = torch.utils.data.DataLoader(DataSet(test_data, normalizer), batch_size=BATCH_SIZE, num_workers=2,pin_memory=True)

data_loader = next(iter(train_loader))
print(len(data_loader["feature"]))
print(len(data_loader["label"]))
#print(data_loader["feature"], "\n", data_loader["label"])

120
120


In [12]:
#Tạo model
class Model(L.LightningModule):
  def __init__(self, num_classes=len(labels), learning_rate=5e-4, input_dim=len(COLUMNS)):
    super(Model, self).__init__()
    self.learning_rate = learning_rate
    self.mlp = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, num_classes),
        )

  def forward(self, x):
    return self.mlp(x)

  def training_step(self, batch, batch_idx):
    x = batch["feature"]
    y = batch["label"]
    y_pred = self(x)
    loss = torch.nn.CrossEntropyLoss()(y_pred, y)
    self.log('train_loss', loss, prog_bar=True)
    return loss

  def test_step(self,batch, batch_idx):
    x = batch["feature"]
    y = batch["label"]
    y_pred = self(y)
    loss = torch.nn.CrossEntropyLoss()(y_pred, y)
    self.log('test_loss', loss, prog_bar = True)
    return loss

  def validation_step(self, batch, batch_idx):
    x = batch["feature"]
    y = batch["label"]
    y_pred = self(x)
    val_loss = torch.nn.CrossEntropyLoss()(y_pred, y)
    self.log('val_loss', val_loss, prog_bar=True)
    return val_loss

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
    return optimizer

In [13]:
#train model
model = Model()

trainer = L.Trainer(max_epochs=50, detect_anomaly=True)
trainer.fit(model, train_loader, val_loader)

INFO: You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
INFO:lightning.pytorch.utilities.rank_zero:You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name | Type       | P

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=50` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.
